<a href="https://colab.research.google.com/github/kevincong95/cs231n-emotiw/blob/master/notebooks/audio/1.0-la-audio_openl3_hyperparam_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function

In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import tensorflow as tf
import tensorflow

In [4]:
print(tf.__version__)


1.15.2


In [5]:
!pip install hyperas

In [6]:
import numpy as np
from tensorflow import keras
from hyperopt import Trials, STATUS_OK, tpe
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils
from hyperas import optim
from hyperas.distributions import choice, uniform

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [0]:
# TO DO

import numpy as np
def data():
    """
    Data providing function:

    This function is separated from create_model() so that hyperopt
    won't reload data for each evaluation run.
    """
    X_train = np.load('/content/drive/My Drive/Machine-Learning-Projects/cs231n-project/notebooks/audio-final/audio-pickle-all-X-openl3-train.pkl', allow_pickle=True)
    Y_train = np.load('/content/drive/My Drive/Machine-Learning-Projects/cs231n-project/notebooks/audio-final/audio-pickle-all-Y-openl3-train.pkl' , allow_pickle=True)

    Y_val = np.load('/content/drive/My Drive/Machine-Learning-Projects/cs231n-project/notebooks/audio-final/audio-pickle-all-Y-openl3-val.pkl' , allow_pickle=True)
    X_val = np.load('/content/drive/My Drive/Machine-Learning-Projects/cs231n-project/notebooks/audio-final/audio-pickle-all-X-openl3-val.pkl' , allow_pickle=True)
    
    x_train, y_train, x_test, y_test = X_train , Y_train , X_val , Y_val

    return x_train, y_train, x_test, y_test


def create_model(x_train, y_train, x_test, y_test):
    """
    Model providing function:

    Create Keras model with double curly brackets dropped-in as needed.
    Return value has to be a valid python dictionary with two customary keys:
        - loss: Specify a numeric evaluation metric to be minimized
        - status: Just use STATUS_OK and see hyperopt documentation if not feasible
    The last one is optional, though recommended, namely:
        - model: specify the model just created so that we can later use it again.
    """

    inputs = keras.Input(shape=[None,6144])

    # CNN Portion
    x =  tf.keras.layers.Conv1D(64, 2, activation='selu')(inputs) # This does convolves on the time domain. I.e. it is NOT time distributed. 
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling1D(pool_size=2, strides=1, padding='valid')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Conv1D(512, 2, activation='selu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling1D(pool_size=2, strides=1, padding='valid')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Conv1D(512, 2, activation='selu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling1D(pool_size=2, strides=1, padding='valid')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    
    # Recurrent Portion
    x = tf.keras.layers.Bidirectional(keras.layers.LSTM(10, return_sequences=True, input_shape=[None, 6144] ,  dropout=0.2 , activation='selu'))(x)
    x = tf.keras.layers.Bidirectional(keras.layers.LSTM(5))(x)
    x = tf.keras.layers.Dense(32 , activation='selu')(x)
    x = tf.keras.layers.Dropout(0.4)(x)    
    outputs = tf.keras.layers.Dense(3 , activation='softmax')(x)

    # Define Hyperparams and Compile
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
      initial_learning_rate={{uniform(1e-6, 1e-4)}},
      decay_steps=10000,
      decay_rate=0.9)
    model = keras.Model(inputs=inputs, outputs=outputs)
    opt = keras.optimizers.Adam(learning_rate=lr_schedule)
    opt = keras.optimizers.Adagrad(learning_rate=lr_schedule)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt , metrics=['accuracy'])

    
    result = model.fit(x_train , y_train , epochs=10 , batch_size=32,  validation_data=[x_test , y_test]) 
    #get the highest validation accuracy of the training epochs
    validation_acc = np.amax(result.history['val_acc']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}




In [0]:
best_run, best_model = optim.minimize(model=create_model,
                                        data=data,
                                        algo=tpe.suggest,
                                        max_evals=5,
                                        trials=Trials() , notebook_name='audio_openl3_hyperparam_tuning')
X_train, Y_train, X_test, Y_test = data()
print("Evalutation of best performing model:")
print(best_model.evaluate(X_test, Y_test))
save_path_final = "'/content/drive/My Drive/Machine-Learning-Projects/cs231n-project/models/openl3-cnn-lstm-hyperparam-tuning.h5"
print("Best performing model chosen hyper-parameters:")
print(best_run)

Streaming output truncated to the last 5000 lines.
2368/2661 [=========================>....]
 - ETA: 0s - loss: 1.1833 - acc: 0.3395

2400/2661 [==========================>...]
 - ETA: 0s - loss: 1.1834 - acc: 0.3396

2464/2661 [==========================>...]
 - ETA: 0s - loss: 1.1863 - acc: 0.3385

2528/2661 [===========================>..]
 - ETA: 0s - loss: 1.1870 - acc: 0.3358

2592/2661 [============================>.]
 - ETA: 0s - loss: 1.1871 - acc: 0.3349

2624/2661 [============================>.]
 - ETA: 0s - loss: 1.1862 - acc: 0.3350
